In [10]:

import os, sys, subprocess, time, toml
import pandas as pd
import numpy as np
from MOASMO_parameter_allbasin_emulator import allbasin_emulator_CV_traintest_and_optimize_2_ann
import run_multiple_paramsets_Derecho
from multiprocessing import Pool
from MOASMO_parameter_allbasin_emulator import *

iter_end = 1 # e.g., iter_end=2 means outputs from iter0 and iter1 will be used to generate new paprameters for iter 2
ncpus = 5
# ncpus = 20
# iterend = 1
numruns = 2

In [11]:

only_checkruns = False

infile_basin_info = f"/glade/work/guoqiang/CTSM_CAMELS/data_mesh_surf/HillslopeHydrology/CAMELS_level1_basin_info.csv"
infile_param_info = '/glade/u/home/guoqiang/CTSM_repos/CTSM_calibration/src/parameter/CTSM_CAMELS_calibparam_2410.csv'
infile_attr_foruse = '/glade/u/home/guoqiang/CTSM_repos/CTSM_calibration/data/camels_attributes_table_TrainModel.csv'
inpath_moasmo = "/glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_emulator"
path_CTSM_case_all = f'/glade/work/guoqiang/CTSM_CAMELS/Calib_HH_emulator'

# trainmode = 'trainbasin' # allbasin; trainbasin
# trainmode = 'allbasin_2err'
trainmode = 'spaceCV1'
# trainmode = 'allbasin_50iter0'
# trainmode = 'allbasin'


########################################################################################################################
# train on what basins?

if trainmode == 'allbasin':
    target_index = np.arange(627)
    suffix = 'LSEnormKGE'
    outpathname = 'LSE_allbasin'
    objfunc = 'normKGE'
elif trainmode == 'spaceCV1':
    test_index = np.arange(0, 627, 5)
    train_index = np.setdiff1d(np.arange(627), test_index)
    target_index = train_index
    outpathname = 'LSE_normNN_normKGECV0'
    suffix = 'LSEnormNNnormKGECV0'
    objfunc = 'normKGE'



In [12]:
infile_basin_info, infile_param_info, infile_attr_foruse, inpath_moasmo, outpathname, path_CTSM_case, iterend, ncpus, suffix, train_index, numruns, objfunc = infile_basin_info, infile_param_info, infile_attr_foruse, inpath_moasmo, outpathname, path_CTSM_case_all, iter_end, ncpus, suffix, train_index, numruns, objfunc

In [13]:
    outpath = f"{inpath_moasmo}/{outpathname}"
    os.makedirs(outpath, exist_ok=True)

    # Load data: same for all iterations
    df_basin_info = pd.read_csv(infile_basin_info)
    df_basin_info.index = np.arange(len(df_basin_info))
    all_index = np.arange(len(df_basin_info))

    test_index = np.setdiff1d(all_index, train_index)

    # information for all basins
    df_param_info = pd.read_csv(infile_param_info)

    file_defa_param = f'{outpath}/camels_ctsm_defa_param_train_{suffix}.csv'
    df_param_defa_train = read_allbasin_defa_params(path_CTSM_case, infile_param_info, file_defa_param, train_index)
    file_defa_param = f'{outpath}/camels_ctsm_defa_param_test_{suffix}.csv'
    df_param_defa_test = read_allbasin_defa_params(path_CTSM_case, infile_param_info, file_defa_param, test_index)

    file_param_lb = f'{outpath}/camels_ctsm_all_param_lb_train_{suffix}.gz'
    file_param_ub = f'{outpath}/camels_ctsm_all_param_ub_train_{suffix}.gz'
    
    df_param_lb_train, df_param_ub_train = load_basin_param_bounds(inpath_moasmo, df_param_defa_train, file_param_lb, file_param_ub, train_index, suffix)
    file_param_lb = f'{outpath}/camels_ctsm_all_param_lb_test_{suffix}.gz'
    file_param_ub = f'{outpath}/camels_ctsm_all_param_ub_test_{suffix}.gz'
    df_param_lb_test, df_param_ub_test = load_basin_param_bounds(inpath_moasmo, df_param_defa_test, file_param_lb, file_param_ub, test_index, suffix)

    
    file_camels_attribute = f'{outpath}/camels_basin_attribute_train_{suffix}.pkl'
    df_att_train = read_camels_attributes(infile_basin_info, file_camels_attribute, train_index)
    file_camels_attribute = f'{outpath}/camels_basin_attribute_test_{suffix}.pkl'
    df_att_test = read_camels_attributes(infile_basin_info, file_camels_attribute, test_index)

    df_att_foruse = pd.read_csv(infile_attr_foruse)
    useattrs = list(df_att_foruse[df_att_foruse['att_Xie2021'].values]['Attribute_text'].values)
    print("The number of attributes used:", len(useattrs))
    print(useattrs)


    # Load data: outputs from each iteration from training basins
    for iter in range(0, iterend):
        file_all_param = f'{outpath}/camels_ctsm_all_param_train_{suffix}_iter{iter}.gz'
        file_all_metric = f'{outpath}/camels_ctsm_all_metric_train_{suffix}_iter{iter}.gz'
        file_all_basinid = f'{outpath}/camels_ctsm_all_basinid_train_{suffix}_iter{iter}.gz'

        df_param_i, df_metric_i, df_basinid_i = load_all_basin_params_metrics(inpath_moasmo, df_param_defa_train,
                                                                              df_basin_info, iter, file_all_param,
                                                                              file_all_metric, file_all_basinid,
                                                                              train_index, suffix)

        df_basinid_i['iter'] = iter

        if iter == 0:
            df_param = df_param_i
            df_metric = df_metric_i
            df_basinid = df_basinid_i
        else:
            df_param = pd.concat([df_param, df_param_i])
            df_metric = pd.concat([df_metric, df_metric_i])
            df_basinid = pd.concat([df_basinid, df_basinid_i])

    df_param.index = np.arange(len(df_param))
    df_metric.index = np.arange(len(df_metric))
    df_basinid.index = np.arange(len(df_basinid))

    index = np.isnan(np.sum(df_metric.values, axis=1) + np.sum(df_param.values, axis=1))
    df_param = df_param[~index]
    df_metric = df_metric[~index]
    df_basinid = df_basinid[~index]

    df_param.index = np.arange(len(df_param))
    df_metric.index = np.arange(len(df_metric))
    df_basinid.index = np.arange(len(df_basinid))

    print('Number of nan samples:', np.sum(index))
    print("Number of original parameter sets:", len(index))
    print("Number of final parameter sets:", len(df_param))


    # One-hot encoding for categorical attributes
    df_att = pd.concat([df_att_train, df_att_test])
    df_att.index = np.arange(len(df_att))
    df_att_use = df_att[useattrs + ["hru_id"]]
    for att in useattrs:
        if df_att_use[att].dtype == "object":
            print('Convert', att, 'to one-hot encoding')
            enc = OneHotEncoder(sparse_output=False)
            enc.fit(df_att_use[[att]])
            encnames = [att + "_" + str(i) for i in range(len(enc.categories_[0]))]
            print('New columns:', encnames)
            df_enc = pd.DataFrame(enc.transform(df_att_use[[att]]), columns=encnames)
            df_att_use = pd.concat([df_att_use, df_enc], axis=1)
            df_att_use = df_att_use.drop([att], axis=1)

    df_att_use_train = df_att_use[:len(df_att_train)]
    df_att_use_test = df_att_use[len(df_att_train):]
    df_att_use_train.index = np.arange(len(df_att_use_train))
    df_att_use_test.index = np.arange(len(df_att_use_test))
    
    useattrs = list(df_att_use_train.columns)
    useattrs.remove('hru_id')

    # Prepare model input and output
    df_input = df_param.copy()
    df_input["hru_id"] = df_basinid["hru_id"]
    df_input = df_input.merge(df_att_use_train[useattrs + ["hru_id"]], on="hru_id", how="left")
    df_input = df_input.drop(["hru_id"], axis=1)

    inputnames = list(df_param.columns) + useattrs
    x_all = df_input[inputnames].values.copy()
    print("Input shape:", x_all.shape)

    print('Train/test model')
    print('Train index:', train_index)

    # Normalize the features
    x_train_mean = np.mean(x_all, axis=0)
    x_train_std = np.std(x_all, axis=0)    
    x_all_scaled = (x_all - x_train_mean) / x_train_std

File exists: /glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_emulator/LSE_normNN_normKGECV0/camels_basin_attribute_train_LSEnormNNnormKGECV0.pkl
File exists: /glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_emulator/LSE_normNN_normKGECV0/camels_basin_attribute_test_LSEnormNNnormKGECV0.pkl
The number of attributes used: 27
['mean_elev', 'mean_slope', 'area_gauges2', 'p_mean', 'pet_mean', 'aridity', 'p_seasonality', 'frac_snow', 'high_prec_freq', 'high_prec_dur', 'low_prec_freq', 'low_prec_dur', 'frac_forest', 'lai_max', 'lai_diff', 'dom_land_cover', 'dom_land_cover_frac', 'soil_depth_pelletier', 'soil_depth_statsgo', 'soil_porosity', 'soil_conductivity', 'max_water_content', 'sand_frac', 'silt_frac', 'clay_frac', 'carbonate_rocks_frac', 'geol_permeability']
Number of nan samples: 1079
Number of original parameter sets: 100200
Number of final parameter sets: 99121
Convert dom_land_cover to one-hot encoding
New columns: ['dom_land_cover_0', 'dom_land_cov